# Import Libraries and Define ROS Initialization


In [1]:
# import libraries and define ros initialization
import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
import assignment_2_2023.msg
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
%matplotlib widget

# ROS Initialization

In [2]:
def ros_components_init():
    global client
    rospy.init_node('set_target_client', anonymous=True)
    client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2023.msg.PlanningAction)
    client.wait_for_server()
    goal_cancelled = False
    pub = rospy.Publisher("/pos_vel", Vel, queue_size=1)

    return pub, client, goal_cancelled

pub, client, goal_cancelled = ros_components_init()


# Widget Initialization and Label Update


In [3]:
# define widget initialization and label update
def init_widgets():
    global goal_status  # Make goal_status global
    pos_label = widgets.Label(value="Position: (0.0, 0.0)")
    vel_label = widgets.Label(value="Velocity: linear_x=0.0, angular_z=0.0")
    target_label = widgets.Label(value="Target: (0.0, 0.0)")
    
    goal_x_input = widgets.FloatText(description="Goal X:")
    goal_y_input = widgets.FloatText(description="Goal Y:")
    set_goal_button = widgets.Button(description="Set New Goal")
    cancel_goal_button = widgets.Button(description="Cancel Goal")
    goal_status = widgets.Text(value="No goal set", description='Goal Status:', disabled=True)  # Added
    
    display(pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status)  # Updated display call
    
    return pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status  # Updated return statement

pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status = init_widgets()


Label(value='Position: (0.0, 0.0)')

Label(value='Velocity: linear_x=0.0, angular_z=0.0')

Label(value='Target: (0.0, 0.0)')

FloatText(value=0.0, description='Goal X:')

FloatText(value=0.0, description='Goal Y:')

Button(description='Set New Goal', style=ButtonStyle())

Button(description='Cancel Goal', style=ButtonStyle())

Text(value='No goal set', description='Goal Status:', disabled=True)

# Publishing Position and Velocity


In [4]:
#publish position and velocity
def publish_position_velocity(msg):
    global pos_label, vel_label, pub
    current_pos = msg.pose.pose.position
    current_vel_linear = msg.twist.twist.linear
    current_vel_angular = msg.twist.twist.angular
    
    pos_and_vel = Vel()
    pos_and_vel.pos_x = current_pos.x
    pos_and_vel.pos_y = current_pos.y
    pos_and_vel.vel_x = current_vel_linear.x
    pos_and_vel.vel_z = current_vel_angular.z
    
    pub.publish(pos_and_vel)
    
    pos_label.value = f"Position: ({current_pos.x}, {current_pos.y})"
    vel_label.value = f"Velocity: linear_x={current_vel_linear.x}, angular_z={current_vel_angular.z}"

rospy.Subscriber("/odom", Odometry, publish_position_velocity)


# Setting and Canceling Goals


In [5]:
# set new goal
def set_new_goal(b):
    global goal_x_input, goal_y_input, goal_status, goal_cancelled  # Added goal_cancelled
    global target_label
    input_x = goal_x_input.value
    input_y = goal_y_input.value
    
    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)
    
    goal = assignment_2_2023.msg.PlanningGoal()
    goal.target_pose.pose.position.x = input_x
    goal.target_pose.pose.position.y = input_y
    client.send_goal(goal, done_cb=goal_done_cb, feedback_cb=goal_feedback_cb)
    goal_cancelled = False  # Reset the flag whenever a new goal is set
    
    target_label.value = f"Target: ({input_x}, {input_y})"
    goal_status.value = f"Goal set at ({input_x}, {input_y})"  # Update goal status

#cancel goal
def cancel_goal(b):
    global goal_cancelled, goal_status  # Added goal_status
    global target_label
    if not goal_cancelled:
        goal_cancelled = True
        client.cancel_goal()
        rospy.loginfo("Current goal has been cancelled")
        goal_status.value = "Goal canceled"  # Update goal status
    else:
        rospy.loginfo("No active goal to cancel")
    
    target_label.value = "Target: (0.0, 0.0)"

set_goal_button.on_click(set_new_goal)
cancel_goal_button.on_click(cancel_goal)


# Handling Goal Results


In [6]:
# handle goal results and feedback
target_reached = 0
target_canceled = 0

def goal_done_cb(state, result):
    global target_reached, target_canceled, goal_status  # Added goal_status
    if state == actionlib.GoalStatus.SUCCEEDED:
        target_reached += 1
        goal_status.value = "Goal reached"
    else:
        target_canceled += 1
        goal_status.value = "Goal failed/canceled"
    vix_goals.update(target_reached, target_canceled)  # Update the bar chart

def goal_feedback_cb(feedback):
    pass


# Visualizing Robot Position in a Separate Window


In [7]:
#plot robot position
class Visualiser:
    def __init__(self):
#         self.root = root
#         self.root.title("Robot Position")
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'bo')
        self.x_data, self.y_data = [], []

#         self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
#         self.canvas.get_tk_widget().pack()

    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.grid(True)  # Add grid lines to the plot
        return self.ln,
        
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)

    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln,


# Visualizing Robot Velocity in a Separate Window


In [8]:
#plot robot velocity
class VelocityVisualiser:
    def __init__(self):
#         self.root = root
#         self.root.title("Robot Velocity")
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'r')
        self.vel_data = []

#         self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
#         self.canvas.get_tk_widget().pack()

    def plot_init(self):
        self.ax.set_xlim(0, 100)
        self.ax.set_ylim(-1, 1)
        self.ax.grid(True)  # Add grid lines to the plot
        return self.ln,
        
    def odom_callback(self, msg):
        linear_vel = msg.twist.twist.linear.x
        self.vel_data.append(linear_vel)
        
        if len(self.vel_data) > 100:
            self.vel_data.pop(0)

    def update_plot(self, frame):
        self.ln.set_data(range(len(self.vel_data)), self.vel_data)
        self.ax.set_xlim(0, len(self.vel_data))
        return self.ln,


# Visualizing Goal Status in a Separate Window


In [9]:
#plot goal status
class VisualizerGoals:
    def __init__(self, root):
        self.root = root
        self.root.title("Goal Status")
        self.fig, self.ax = plt.subplots()
        self.res = ['Successful', 'Failed']
        self.n_goals = [0, 0]
        self.bar_colors = ['tab:green', 'tab:red']
        self.bar = self.ax.bar(self.res, self.n_goals, color=self.bar_colors)
        self.ax.set_title('Counting the State of the Goals - Successful or Failed')
        self.ax.set_ylim(0, 10)  # Adjust this to a reasonable upper limit based on expected number of goals

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.get_tk_widget().pack()

    def update(self, s, c):
        self.n_goals = [s, c]
        for bar, height in zip(self.bar, self.n_goals):
            bar.set_height(height)
        self.ax.relim()
        self.ax.autoscale_view()
        self.fig.canvas.draw()


# Cell 10: Main Function


In [10]:
#main function
if __name__ == "__main__":
#     pos_label, vel_label, target_label, goal_x_input, goal_y_input, set_goal_button, cancel_goal_button, goal_status = init_widgets()  # Updated to unpack 8 values
    
    rospy.Subscriber("/odom", Odometry, publish_position_velocity)

    root_position = tk.Tk()
    vis = Visualiser()
    sub1 = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
    ani1 = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)

    root_velocity = tk.Tk()
    vel_vis = VelocityVisualiser()
    sub2 = rospy.Subscriber('/odom', Odometry, vel_vis.odom_callback)
    ani2 = FuncAnimation(vel_vis.fig, vel_vis.update_plot, init_func=vel_vis.plot_init)

    root_goals = tk.Tk()
    vix_goals = VisualizerGoals(root_goals)
    
    set_goal_button.on_click(set_new_goal)
    cancel_goal_button.on_click(cancel_goal)
    
#     root_position.mainloop()
#     root_velocity.mainloop()
#     root_goals.mainloop()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …